# Clustering Assingment

Hi,

and welcome to our clustering assignment. We're absolutely excited to have you getting your hands dirty on this exercise.

Here's the situation:

We've received geographical data from a company which resembles their sales territories. The customer has 4 sales territories and asks us to:

- create one geographical structure where each territory is cut into four sub-territories, whereas each of the created sub-territories should be geographically united, and of equal size in population.
- create another geographical structure where each territory is cut into as many sub-territories as possible, whereas each of the created sub-territories should be geographically united, and each sub-territory should have a population of at least 500000.

Here is what your colleague has prepared for:

## Data sources

The dataset stems from a public data source that provides administrative Community boundaries of Germany with population number.

Find a detailed data set description here:
https://opendata-esri-de.opendata.arcgis.com/datasets/esri-de-content::gemeindegrenzen-2019-mit-einwohnerzahl/data?geometry=8.643%2C50.790%2C10.888%2C51.093

Let's load the data first.

#### data sources: load the raw data

In [ ]:
# in case you need to install
#!pip install geopandas

In [ ]:
import geopandas as gpd
url = "https://opendata.arcgis.com/datasets/662209f48e384845974d2cff8d8c2fbd_0.geojson"
gdf = gpd.read_file(url)

In [ ]:
# add community_code
ags_cols = ['SN_L', 'SN_R', 'SN_K','SN_V1', 'SN_V2', 'SN_G']
gdf['community_code'] = gdf.SN_L.astype(str)+gdf.SN_R.astype(str)+gdf.SN_K.astype(str)+gdf.SN_V1.astype(str)+gdf.SN_V2.astype(str)+gdf.SN_G.astype(str)
gdf['community_code'] = gdf['community_code'].astype(int)

#### data sources: sales territories

In [ ]:
# prepare sales territories
import numpy as np
gdf.loc[gdf.SN_L.isin(['01', '02', '03', '04', '13']), 'sales_territory'] = 'north'
gdf.loc[gdf.SN_L.isin(['06', '08', '09']), 'sales_territory'] = 'south'
gdf.loc[gdf.SN_L.isin(['05', '07', '10']), 'sales_territory'] = 'west'
gdf.loc[gdf.SN_L.isin(['11','12', '14', '15','16']), 'sales_territory'] = 'east'

sales_terr = gdf[['community_code', 'sales_territory']].copy()

sales_terr.head()

#### data sources: geographies

In [ ]:
# add centroids per community code
def getCentroids(geoDF):
    ctr_pts = [geoDF.geometry[i][0].centroid for i in range(len(geoDF))]
    return ctr_pts
        
def getXY(ctr_pts):
    return (ctr_pts.x, ctr_pts.y)

gdf["longitude_centroid"], gdf["latitude_centroid"] = [list(t) for t in zip(*map(getXY, getCentroids(gdf)))]

geographies = gdf[["community_code","longitude_centroid","latitude_centroid", 'SHAPE_Length', 'SHAPE_Area', 'geometry']].copy()

geographies.head()

#### data sources: reference data

In [ ]:
ref_cols = [
    'community_code', 'GEN', 'BEZ', 'IBZ', 'BEM',
    'AGS', 'SN_L', 'SN_R', 'SN_K','SN_V1', 'SN_V2', 'SN_G', 
    'FK_S3', 'NUTS', 'WSK', 'EWZ', 'KFL'
]
ref_data = gdf[ref_cols].copy()

rename_ref_dict = {
    'GEN':'community_name',
    'BEZ':'community_type',
    'BEZ':'community_association_class',
    'SN_L':'federal_state_subcode',
    'SN_R':'administrative_district_subcode', 
    'SN_K':'county_subcode',
    'SN_V1':'municipal_association_subcode',
    'SN_V2':'municipal_local_association_subcode',
    'SN_G':'community_subcode', 
    'EWZ':'population', 
    'KFL':'area_in_square_km',
}
ref_data = ref_data.rename(columns=rename_ref_dict)

ref_data.head()

#### data model

Your colleague has done even a little more preparation work and created a diagram of the resulting data model.

Find the diagram here: https://dbdiagram.io/d/611b77312ecb310fc3ce7f7f

#### inspect the data by plotting with folum

Here's another goodie from your colleague: a visualisation of the data done with the help of folium (reference: https://geopandas.readthedocs.io/en/latest/gallery/polygon_plotting_with_folium.html).

In [ ]:
# incase you need to install folium
#!pip install folium

In [ ]:
import folium
m = folium.Map(location=[49.8437, 9.1613], zoom_start=9, tiles='CartoDB positron')

# Overlay the boundaries of boroughs on map with borough name as popup
for _, r in geographies.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001) #without simplifying the representation of each borough, the map might not be displayed
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'lightblue'})
    folium.Popup(f"community code: {r['community_code']}").add_to(geo_j)
    geo_j.add_to(m)
m

## expected outcome

Create a geographical structure 'sales_terr_1' where each territory is cut into four sub-territories, whereas each of the created sub-territories should be geographically united, and of equal size in population.

The dataframe below provides the basic structure of the output format. Now, the sales_subterritory column is empty. Fill the column with your territory clusters.

In [ ]:
sales_terr_1 = sales_terr.merge(ref_data[['community_code','population']], on='community_code')
sales_terr_1['sales_subterritory'] = None
sales_terr_1 = sales_terr_1[['community_code', 'sales_territory','sales_subterritory','population']]
sales_terr_1.head()

Create another geographical structure 'sales_terr_2' where each territory is cut into as many sub-territories as possible, whereas each of the created sub-territories should be geographically united, and each sub-territory should have a population of at least 500000.

The dataframe below provides the basic structure of the output format. Now, the sales_subterritory column is empty. Fill the column with your territory clusters.

In [ ]:
sales_terr_2 = sales_terr.merge(ref_data[['community_code','population']], on='community_code')
sales_terr_2['sales_subterritory'] = None
sales_terr_2 = sales_terr_2[['community_code', 'sales_territory','sales_subterritory','population']]
sales_terr_2.head()

In case you should have questions, reach out to Siggi: siegstedt@machinemind.io
    
We're excited to see your results.